# 2.3 Symbolic Data

In this section we extend the representational capability of our language by introducing the ability to work with arbitrary symbols as data.

## 2.3.1 Quotation

$(a\ b) \neq (list\ a\ b)$ 

In [1]:
(define a 1)
(define b 2)
(define c 3)

In [2]:
(list a b c )

(1 2 3)

In [3]:
(list 'a 'b c)

(a b 3)

In [4]:
(car '(a b c))

a

In [5]:
(cdr '(a b c))

(b c)

In [6]:
(eval (car '(a b)))

1

In [7]:
(eq? 1 a)

#t

In [8]:
(eq? 1 'a)

#f

In [9]:
; memq:
; arguments - symbol
;           - list
; If the symbol is not contained in the list, then 'memq' returns false

(define (memq item x)
  (cond ((null? x) #f)
        ((eq? item (car x)) x)
        (else (memq item (cdr x)))))

In [10]:
(memq 'apple '(pepper banana prune))

#f

In [11]:
(memq 'apple '(x (apple sauce) y apple pear))

(apple pear)

In [12]:
(memq 1 '(a b c))

#f

In [13]:
(memq 'a '(a b c))

(a b c)

### Exercise 2.53

```lisp
1. (list 'a 'b 'c)
2. (list (list 'george))
3. (cdr '((x1 x2) (y1 y2)))
4. (cadr '((x1 x2) (y1 y2)))
5. (pair? (car '(a short list)))
6. (memq 'red '((red shoes) (blue socks)))
7. (memq 'red '(red shoes blue socks))
```
1. (a b c)
2. ((george))
3. ((y1 y2))
4. (y1 y2)
5. #f
6. #f
7. (red hoes blue socks)

In [14]:
(list 'a 'b 'c)

(a b c)

In [15]:
(list (list 'george))

((george))

In [16]:
(cdr '((x1 x2) (y1 y2)))

((y1 y2))

In [17]:
(cadr '((x1 x2) (y1 y2)))

(y1 y2)

In [18]:
(pair? (car '(a short list)))

#f

In [19]:
(memq 'red '((red shoes) (blue socks)))

#f

In [20]:
(memq 'red '(red shoes blue socks))

(red shoes blue socks)

### Exercise 2.54

In [21]:
(equal? '(this is a list) '(this is a list))

#t

In [22]:
(equal? '(this (is a) list) '(this is a list))

#f

In [23]:
(equal? '(this a is list) '(this is a list))

#f

In [24]:
;answer

(define (my-equal? xs ys)
  (cond ((or (null? xs) (null? ys)) #t)
        ((eq? (car xs) (car ys)) (my-equal? (cdr xs) (cdr ys)))
        (else #f)))

In [25]:
(my-equal? '(this is a list) '(this is a list))

#t

In [26]:
(my-equal? '(this is a list) '(this (is a) list))

#f

In [27]:
(my-equal? '(this a is list) '(this is a list))

#f

### Exercise 2.55

In [28]:
(car ''abracadabra)

quote

In [29]:
''abracadabra

(quote abracadabra)

In [30]:
(quasiquote abracadabra)

abracadabra

In [31]:
(quasiquote (quasiquote abracadabra))

(quasiquote abracadabra)

In [32]:
(eq? 'abracadabra (quasiquote abracadabra))
; =
;(eq? 'abracadabra (quote abracadabra))

#t

`symbol　は　(quasiquote symbol)　のシンタックスシュガー

## 2.3.2 Example: Symbolic Differentiation

In [33]:
(define (variable? e)
  (symbol? e))

(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))

(define (sum? e)
  (and (pair? e) (eq? (car e) `+)))

(define (addend e)
  (cadr e))

(define (augend e)
  (caddr e))

;(define (make-sum a1 a2)
;  (list `+ a1 a2))

;modify & add a function '=number?'
(define (make-sum a1 a2)
  (cond ((=number? a1 0) a2)
        ((=number? a2 0) a1)
        ((and (number? a1) (number? a2))
         (+ a1 a2))
        (else (list '+ a1 a2))))

(define (=number? exp num)
  (and (number? exp) (= exp num)))


(define (product? e)
  (and (pair? e) (eq? (car e) `*)))

(define (multiplier e)
  (cadr e))

(define (multiplicand e)
  (caddr e))

;(define (make-product m1 m2)
;  (list `* m1 m2))

; modify
(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
        ((=number? m1 1) m2)
        ((=number? m2 1) m1)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list '* m1 m2))))

In [34]:
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        ((product? exp) (make-sum
                             (make-product (multiplier exp)
                                           (deriv (multiplicand exp) var))
                             (make-product (deriv (multiplier exp) var)
                                           (multiplicand exp))))
        (else (error "unknown expression type: DERIV" "hoge"))))

In [35]:
(deriv '(+ x 3) 'x)

1

In [36]:
(deriv '(* x y) 'x)

y

In [37]:
(deriv '(* (* x y) (+ x 3)) 'x)

(+ (* x y) (* y (+ x 3)))

### Exercise 2.56

In [38]:
(define (exponentiation? v)
  (and (pair? v)
       (eq? '** (car v))))

(define (base v)
  (cadr v))

(define (exponent v)
  (caddr v))

(define (make-exponentiation b e)
  (cond ((=number? e 1) b)
        ((=number? e 0) 1)
        (else (list '** b e))))


In [39]:
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        ((product? exp) (make-sum
                             (make-product (multiplier exp)
                                           (deriv (multiplicand exp) var))
                             (make-product (deriv (multiplier exp) var)
                                           (multiplicand exp))))
        ((exponentiation? exp) (make-product (exponent exp)
                                             (make-product (make-exponentiation (base exp) (- (exponent exp) 1))
                                                           (deriv (base exp) var))))
        (else (error "unknown expression type: DERIV" "error"))))

In [40]:
(deriv '(** x 5) 'x)

(* 5 (** x 4))

In [41]:
(make-exponentiation 'x 0)

1

In [42]:
(make-exponentiation 'x 1)

x

### Exercise 2.57

In [43]:
(define (augend e)
  (if (null? (cdddr e))
      (caddr e)
      (cons '+ (cddr e))))


(define (multiplicand e)
  (if (null? (cdddr e))
      (caddr e)
      (cons '* (cddr e))))

;;
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        ((product? exp) (make-sum
                             (make-product (multiplier exp)
                                           (deriv (multiplicand exp) var))
                             (make-product (deriv (multiplier exp) var)
                                           (multiplicand exp))))
        ((exponentiation? exp) (make-product (exponent exp)
                                             (make-product (make-exponentiation (base exp) (- (exponent exp) 1))
                                                           (deriv (base exp) var))))
        (else (error "unknown expression type: DERIV" "error"))))

In [44]:
(deriv '(* x (* y (+ x 3))) 'x)

(+ (* x y) (* y (+ x 3)))

In [45]:
(deriv '(* x y (+ x 3)) 'x)

(+ (* x y) (* y (+ x 3)))

In [46]:
(product? (multiplicand '(* x y (+ x 3))))

#t

In [47]:
(multiplicand (multiplicand '(* x y (+ x 3))))

(+ x 3)

In [48]:
(cdddr '(* y (+ x 3)))

()

### Exercise 2.58

In [49]:
; a.


(define (variable? e)
  (symbol? e))

(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))

(define (sum? e)
  (and (pair? e) (eq? (cadr e) `+)))

(define (addend e)
  (car e))

(define (augend e)
  (caddr e))

;(define (make-sum a1 a2)
;  (list  a1 '+ a2))

;modify & add a function '=number?'
(define (make-sum a1 a2)
  (cond ((=number? a1 0) a2)
        ((=number? a2 0) a1)
        ((and (number? a1) (number? a2))
         (+ a1 a2))
        (else (list  a1 '+ a2))))

(define (=number? exp num)
  (and (number? exp) (= exp num)))


(define (product? e)
  (and (pair? e) (eq? (cadr e) `*)))

(define (multiplier e)
  (car e))

(define (multiplicand e)
  (caddr e))

;(define (make-product m1 m2)
;  (list  m1 '* m2))

; modify
(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
        ((=number? m1 1) m2)
        ((=number? m2 1) m1)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list  m1 '* m2))))


In [50]:
;;
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        ((product? exp) (make-sum
                             (make-product (multiplier exp)
                                           (deriv (multiplicand exp) var))
                             (make-product (deriv (multiplier exp) var)
                                           (multiplicand exp))))
        ((exponentiation? exp) (make-product (exponent exp)
                                             (make-product (make-exponentiation (base exp) (- (exponent exp) 1))
                                                           (deriv (base exp) var))))
        (else (error "unknown expression type: DERIV" "error"))))

In [51]:
(deriv '(x + (3 * (x + (y + 2)))) 'x)

4

In [52]:
; b.

(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((product? exp) (make-sum
                             (make-product (multiplier exp)
                                           (deriv (multiplicand exp) var))
                             (make-product (deriv (multiplier exp) var)
                                           (multiplicand exp))))
        ((exponentiation? exp) (make-product (exponent exp)
                                             (make-product (make-exponentiation (base exp) (- (exponent exp) 1))
                                                           (deriv (base exp) var))))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        (else (error "unknown expression type: DERIV" "error"))))


(define (augend e)
  (if (null? (cdddr e))
      (caddr e)
      (cond ((product? (cddr e)) (cddr e))
            ((sum? (cddr e)) (cddr e)))))


(define (multiplicand e)
  (if (null? (cdddr e))
      (caddr e)
      (cond ((product? (cddr e)) (cddr e))
            ((sum? (cddr e)) (cddr e)))))


In [53]:
(deriv '(x + 3 * (x + y + 2)) 'x)

4

In [54]:
(deriv '(x +(x + y + 2) * 3) 'x)

4

## 2.3.3 Example: Representing Sets